In [27]:
import cv2
import csv
from tkinter import Tk, filedialog
import numpy as np  # Needed for rotation matrix computations
from numpy import cos, sin

def select_rois_and_save_to_csv():

    # Initialize Tkinter and hide the root window
    root = Tk()
    root.withdraw()

    # Open a file dialog to select the video file
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi")])
    if not video_path:
        print("No video file selected.")
        return
    
    # Define the output CSV file
    output_csv = video_path.rsplit('.', 1)[0] + '_ROIs.csv'

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Unable to open the video file.")
        return

    # Read the first frame
    ret, first_frame = cap.read()
    cap.release()
    if not ret:
        print("Error: Unable to read the first frame.")
        return

    # Initialize variables
    clone = first_frame.copy()
    roi = []  # Current ROI [(x1, y1), (x2, y2)]
    dragging = [False]  # For moving ROI
    drag_start = None
    saved_rois = []
    current_angle = [0]  # Current angle for rotation
    rotate_factor = 2  # Amount of change per scroll
    resize_factor = 2  # Amount of change per scroll

    # Mouse callback function
    def handle_mouse(event, x, y, flags, param):
        nonlocal roi, clone, drag_start, saved_rois, current_angle, rotate_factor, resize_factor

        # Start drawing the rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            if not dragging[0]:
                dragging[0] = True
                roi = [(x, y)]  # Start new ROI at the clicked position

        # Update rectangle during drawing
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(roi) == 1:
            clone = first_frame.copy()
            for r in saved_rois:
                rotated_rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2, r[4])
            rotated_rectangle(clone, roi[0], (x, y), (0, 255, 255), 2, current_angle)

        # Finish drawing the rectangle
        elif event == cv2.EVENT_LBUTTONUP:
            if dragging[0]:
                dragging[0] = False
                roi.append((x, y))  # End point

        # Start moving the rectangle
        elif event == cv2.EVENT_RBUTTONDOWN and len(roi) == 2:
            dragging[0] = True
            drag_start = (x, y)

        # Move the rectangle
        elif event == cv2.EVENT_MOUSEMOVE and dragging[0] and len(roi) == 2:
            dx = x - drag_start[0]
            dy = y - drag_start[1]
            drag_start = (x, y)
            roi[0] = (roi[0][0] + dx, roi[0][1] + dy)
            roi[1] = (roi[1][0] + dx, roi[1][1] + dy)
            clone = first_frame.copy()
            for r in saved_rois:
                rotated_rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2, r[4])
            rotated_rectangle(clone, roi[0], roi[1], (0, 255, 255), 2, current_angle)

        # Stop moving the rectangle
        elif event == cv2.EVENT_RBUTTONUP and len(roi) == 2:
            dragging[0] = False

        # Resize or rotate the ROI using scroll wheel
        elif event == cv2.EVENT_MOUSEWHEEL and len(roi) == 2:
            if flags & cv2.EVENT_FLAG_CTRLKEY:  # Resize with Ctrl key pressed
                x1, y1 = roi[0]
                x2, y2 = roi[1]
                if flags > 0:  # Scroll up
                    x1 -= x1*0.01
                    y1 -= y1*0.01
                    x2 += x2*0.01
                    y2 += y2*0.01
                else:  # Scroll down
                    x1 += x1*0.01
                    y1 += y1*0.01
                    x2 -= x2*0.01
                    y2 -= y2*0.01
                roi = [(x1, y1), (x2, y2)]
            else:  # Rotate without modifier key
                if flags > 0:  # Scroll up
                    current_angle[0] -= rotate_factor
                else:  # Scroll down
                    current_angle[0] += rotate_factor

            clone = first_frame.copy()
            for r in saved_rois:
                rotated_rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2, r[4])
            rotated_rectangle(clone, roi[0], roi[1], (0, 255, 255), 2, current_angle)

            # Draw the updated ROI and display height, width, and angle
        if len(roi) == 2:
            x1, y1 = roi[0]
            x2, y2 = roi[1]
            width = abs(x2 - x1)
            height = abs(y2 - y1)
            angle = current_angle[0]

            # Display height, width, and angle at the bottom of the frame
            text = f"Width: {width}, Height: {height}, Angle: {angle}°"
            font_scale = 0.5
            font_thickness = 1
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
            text_x = 10
            text_y = first_frame.shape[0] - 10
            cv2.rectangle(clone, (text_x - 5, text_y - text_size[1] - 5), 
                        (text_x + text_size[0] + 5, text_y + 5), (0, 0, 0), -1)  # Background for text
            cv2.putText(clone, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 
                        font_scale, (255, 255, 255), font_thickness)
        

    def rotated_rectangle(image, start_point, end_point, color, thickness, rotation=0):
        center_point = [(start_point[0]+end_point[0])//2, (start_point[1]+end_point[1])//2]
        height = end_point[1] - start_point[1]
        width = end_point[0] - start_point[0]
        angle = np.radians(rotation)

        # Determine the coordinates of the 4 corner points
        rotated_rect_points = []
        x = center_point[0] + ((width / 2) * cos(angle)) - ((height / 2) * sin(angle))
        y = center_point[1] + ((width / 2) * sin(angle)) + ((height / 2) * cos(angle))
        rotated_rect_points.append([x,y])
        x = center_point[0] - ((width / 2) * cos(angle)) - ((height / 2) * sin(angle))
        y = center_point[1] - ((width / 2) * sin(angle)) + ((height / 2) * cos(angle))
        rotated_rect_points.append([x,y])
        x = center_point[0] - ((width / 2) * cos(angle)) + ((height / 2) * sin(angle))
        y = center_point[1]- ((width / 2) * sin(angle)) - ((height / 2) * cos(angle))
        rotated_rect_points.append([x,y])
        x = center_point[0] + ((width / 2) * cos(angle)) + ((height / 2) * sin(angle))
        y = center_point[1] + ((width / 2) * sin(angle)) - ((height / 2) * cos(angle))
        rotated_rect_points.append([x,y])
        cv2.polylines(image, np.array([rotated_rect_points], np.int32), True, color, thickness)

    # Set up the window and mouse callback
    cv2.namedWindow("Select ROIs")
    cv2.setMouseCallback("Select ROIs", handle_mouse)

    print("Instructions:")
    print("1. Left-click and drag to draw a rectangle.")
    print("2. Right-click and drag to move the rectangle.")
    print("3. Use the scroll wheel to resize the rectangle.")
    print("4. Press 'S' to save the current rectangle to the image.")
    print("5. Press 'Q' to quit and save all rectangles to the CSV file.")

    while True:
        cv2.imshow("Select ROIs", clone)
        key = cv2.waitKey(1) & 0xFF
            
        if key == ord('s') and len(roi) == 2:  # Save the ROI
            x1, y1 = roi[0]
            x2, y2 = roi[1]
            width = abs(x2 - x1)
            height = abs(y2 - y1)
            saved_roi = (min(x1, x2), min(y1, y2), width, height, current_angle[0])
            saved_rois.append(saved_roi)
            print(f"Saved ROI: {saved_roi}")

            clone = first_frame.copy()
            for r in saved_rois:
                rotated_rectangle(clone, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 255, 0), 2, r[4])

        elif key == ord('q'):  # Quit and save
            print("Exiting and saving...")
            break

    cv2.destroyAllWindows()

    # Save the ROIs to a CSV file
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['x', 'y', 'width', 'height', 'angle'])
        writer.writerows(saved_rois)

    print(f"ROIs saved to {output_csv}")




In [28]:
# Example usage:
select_rois_and_save_to_csv()

Instructions:
1. Left-click and drag to draw a rectangle.
2. Right-click and drag to move the rectangle.
3. Use the scroll wheel to resize the rectangle.
4. Press 'S' to save the current rectangle to the image.
5. Press 'Q' to quit and save all rectangles to the CSV file.
Saved ROI: (175, 166, 189, 134, 0)
Saved ROI: (175, 166, 189, 134, -40)
Saved ROI: (176, 167, 185, 127, -90)
Exiting and saving...
ROIs saved to C:/Users/dhers/Desktop/NOR/Chopped videos/3xTg_15mth_1_A_L_ROIs.csv
